# Historical Data

In [ ]:
import pandas as pd

In [417]:
historical_data = pd.read_csv('/Users/monmac/Desktop/448f97988afe977af36cfc73cde95211.csv')

In [459]:
historical_data.isnull().sum();

In [437]:
pd.to_datetime(1629356400, unit='s')

Timestamp('2021-08-19 07:00:00')

In [421]:
historical_data['rain_1h'].isnull().sum()

90637

In [458]:
historical_data['weather_main'].value_counts();

In [423]:
possible_conditions = ['Clear', 'Clouds', 'Rain', 'Mist', 'Drizzle', 'Fog', 'Snow', 'Thunderstorm', 'Haze']

In [457]:
historical_data['weather_description'];

In [456]:
historical_data.dtypes;

In [441]:
historical_data['dt'] = pd.to_datetime(historical_data['dt'], unit='s')
historical_data = historical_data.set_index('dt')

In [452]:
lunch = historical_data[historical_data.index[:].hour==12]
dinner = historical_data[historical_data.index[:].hour==18]
historical_services = pd.concat([lunch, dinner])
historical_services = historical_services.sort_values('dt')

In [460]:
historical_services;

#   Forecast using API

In [2]:
import requests
import pandas as pd
def fetch_data(api,city,nodays):
    """
    Get weather data from weather API. Returns empty string if data not found'

    """
    #api = '7aa7fc7be1af4500882144440212208'
    #city = 'Paris'
    #nodays = '7'
    url = f'http://api.weatherapi.com/v1/forecast.json?key={api}&q={city}&days={nodays}&aqi=no&alerts=no'

    response = requests.get(url)
    
    if response.status_code != 200:
        return ''
    data = response.json()
    return data['forecast']['forecastday']

In [3]:
api = '09e033659f124258acf74721212408'
city = 'Paris'
nodays = '3'
url = f'http://api.weatherapi.com/v1/forecast.json?key={api}&q={city}&days={nodays}&aqi=no&alerts=no'

response = requests.get(url)        

In [4]:
response

<Response [200]>

In [5]:
data=fetch_data('09e033659f124258acf74721212408','Paris','3')

In [6]:
data

[{'date': '2021-08-24',
  'date_epoch': 1629763200,
  'day': {'maxtemp_c': 27.2,
   'maxtemp_f': 81.0,
   'mintemp_c': 14.7,
   'mintemp_f': 58.5,
   'avgtemp_c': 20.7,
   'avgtemp_f': 69.2,
   'maxwind_mph': 14.5,
   'maxwind_kph': 23.4,
   'totalprecip_mm': 0.0,
   'totalprecip_in': 0.0,
   'avgvis_km': 10.0,
   'avgvis_miles': 6.0,
   'avghumidity': 58.0,
   'daily_will_it_rain': 0,
   'daily_chance_of_rain': 0,
   'daily_will_it_snow': 0,
   'daily_chance_of_snow': 0,
   'condition': {'text': 'Partly cloudy',
    'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
    'code': 1003},
   'uv': 5.0},
  'astro': {'sunrise': '06:56 AM',
   'sunset': '08:48 PM',
   'moonrise': '10:05 PM',
   'moonset': '08:49 AM',
   'moon_phase': 'Waning Gibbous',
   'moon_illumination': '83'},
  'hour': [{'time_epoch': 1629756000,
    'time': '2021-08-24 00:00',
    'temp_c': 17.2,
    'temp_f': 63.0,
    'is_day': 0,
    'condition': {'text': 'Partly cloudy',
     'icon': '//cdn.weatherapi.com/w

In [7]:
daily_data = pd.DataFrame(data[0]['day']).iloc[2,:]
pd.DataFrame(daily_data).head(3)

,text
maxtemp_c,27.2
maxtemp_f,81.0
mintemp_c,14.7


In [8]:
### gives the weather hour by hour for the entire first day###

daily_hourly_data = pd.DataFrame(data[0]['hour'])
#condition has a code and an image, the code below only keeps the needed weather state
for i in range(len(daily_hourly_data)):
    daily_hourly_data['condition'][i]=daily_hourly_data['condition'][i]['text']

/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_75382/332607310.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_hourly_data['condition'][i]=daily_hourly_data['condition'][i]['text']


In [9]:
#condition is given as three values this keeps only one
data_series=[]
for i in range(3):
    data_series.append(data[i]['hour'])

#condition has a code and an image, the code below only keeps the needed weather state
for x in range(3):
    for y in range(24):
        data_series[x][y]['condition']=data_series[x][y]['condition']['text']

d={}
for i in range(len(data_series)):
    d[i] = pd.DataFrame(data_series[i])

seven_day_forecast = pd.concat([d[i] for i in range(len(data_series))])

In [10]:
seven_day_forecast=seven_day_forecast.reset_index().drop(columns='index')

In [11]:
#convert time column to datetime
seven_day_forecast['time']=pd.to_datetime(seven_day_forecast['time'])

In [12]:
seven_day_forecast['condition'].value_counts()

Partly cloudy    55
Sunny            12
Overcast          4
Cloudy            1
Name: condition, dtype: int64

In [13]:
pd.read_csv('/Users/monmac/Desktop/weather_conditions.csv');

In [14]:
Clear = ['Clear', 'Sunny']
Clouds = ['Clouds','Partly cloudy', 'Cloudy', 'Overcast']
Rain = ['Rain','Patchy rain possible', 'Patchy light rain', 'Light rain', 'Moderate rain at times','Moderate rain','Heavy rain at times','Heavy rain','Moderate or heavy rain shower','Torrential rain shower']
Mist = ['Mist']
Drizzle = ['Drizzle','Patchy light drizzle','Light drizzle','Freezing drizzle','Heavy freezing drizzle','Light rain shower']
Fog = ['Fog', 'Freezing Fog']
Snow = ['Snow','Patchy snow possible', 'Patchy sleet possible', 'Patchy freezing drizzle possible', 'Blowing snow','Blizzard','Moderate or heavy freezing rain','Light freezing rain','Light sleet', 'Moderate or heavy sleet','Patchy light snow','Light snow','Patchy moderate snow','Moderate snow','Patchy heavy snow','Heavy snow','Ice pellets','Light sleet showers', 'Moderate or heavy sleet showers','Light snow showers','Moderate or heavy snow showers','Light showers of ice pellets','Moderate or heavy showers of ice pellets']
Thunderstorm = ['Thunderstorm','Thundery outbreaks possible', 'Patchy light rain with thunder','Moderate or heavy rain with thunder','Patchy light snow with thunder','Moderate or heavy snow with thunder']
Haze = ['Haze']

In [15]:
summed_conditions = [Clear, Clouds, Rain, Mist, Drizzle, Fog, Snow, Thunderstorm, Haze]

In [16]:
#boolean indexing to decrease possibilities of weather condition
for i in range(len(seven_day_forecast['condition'])):
    for x in summed_conditions: 
        if seven_day_forecast['condition'][i] in x: 
            seven_day_forecast['condition'][i]=x[0]

/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_75382/3313053615.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seven_day_forecast['condition'][i]=x[0]


In [19]:
lunch = seven_day_forecast[seven_day_forecast['time'].dt.hour==12]
lunch['Service']='Lunch'
dinner = seven_day_forecast[seven_day_forecast['time'].dt.hour==18]
dinner['Service']='Dinner'

forecasted_services = pd.concat([lunch, dinner])
forecasted_services['time'] = forecasted_services['time'].dt.date


forecasted_services=forecasted_services.set_index('time')
forecasted_services = forecasted_services.sort_values('time')

/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_75382/944811383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lunch['Service']='Lunch'
/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_75382/944811383.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dinner['Service']='Dinner'


In [20]:
forecasted_services

,time_epoch,temp_c,temp_f,is_day,condition,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,...,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,Service
time,,,,,,,,,,,,,,,,,,,,,
2021-08-24,1629799200,25.9,78.6,1,Clear,13.4,21.6,58,ENE,1024.0,...,0,0,0,0,10.0,6.0,15.4,24.8,7.0,Lunch
2021-08-24,1629820800,26.0,78.8,1,Clear,13.0,20.9,52,NE,1022.0,...,0,0,0,0,10.0,6.0,15.0,24.1,7.0,Dinner
2021-08-25,1629885600,23.2,73.8,1,Clouds,11.6,18.7,51,NE,1021.0,...,0,0,0,0,10.0,6.0,13.4,21.6,6.0,Lunch
2021-08-25,1629907200,23.2,73.8,1,Clouds,12.1,19.4,44,NE,1019.0,...,0,0,0,0,10.0,6.0,14.3,23.0,6.0,Dinner
2021-08-26,1629972000,19.4,66.9,1,Clouds,9.4,15.1,0,N,1020.0,...,0,0,0,0,10.0,6.0,10.7,17.3,5.0,Lunch
2021-08-26,1629993600,20.1,68.2,1,Clear,12.5,20.2,12,NNE,1019.0,...,0,0,0,0,10.0,6.0,14.5,23.4,6.0,Dinner


# Historical Forecast